# Importing libraries

In [0]:
import pandas as pd
import pyarrow.parquet as pq

# Read in the data and combine all months into one dataframe
Note: This step does not make sense before the datatime attributes are properly cleaned (e.g. data of May in January)

In [1]:
# function that reads in a parquet file (based on month) and returns a pandas dataframe
def read_trips(path, month):
    mon = month
    if mon < 10:
        mon = '0' + str(mon)
    else:
        mon = str(mon)
    trips = pq.read_table(path + mon + '.parquet')
    trips = trips.to_pandas()
    return trips


# concatenate all of the month-based dataframes into one dataframe - import all trips = pq.read_table('./data/test/yellow_tripdata_2022-xx.parquet')
all_trips = pd.DataFrame()
for i in range(12):
    all_trips = pd.concat([all_trips, read_trips('./data/yellow_tripdata_2022-', i + 1)])

original_data_length = all_trips.shape[0]
# keep copy of original data
all_trips_original = all_trips.copy()

# Drop duplicates and unrealistic trip distances entries (<=0, >70)

In [2]:
# save current length of dataframe
current_length = all_trips.shape[0]


# check the shape of the dataframe and first 5 rows
print(all_trips_original.shape)
print(all_trips_original.head())

# remove duplicates
all_trips.drop_duplicates(inplace=True)

print('after removing duplicates')
# check how many rows were removed
print('Removed rows: ', original_data_length - all_trips.shape[0])
# in percent
print('Removed rows %: ', (original_data_length - all_trips.shape[0]) / original_data_length * 100, '%')

# check the shape of the dataframe
print(all_trips.shape)

# save current length of dataframe
current_length = all_trips.shape[0]

# filter trips with trip_distance > 0 and <=70
# 35 miles is the maximum distance between two points in New York City,
# so from a practical point of view, we can assume that the maximum distance of a taxi ride is 70 miles
all_trips = all_trips[(all_trips['trip_distance'] > 0) & (all_trips['trip_distance'] <= 70)]

# check the shape of the dataframe - how many rows were removed?
print('after removing unrealistic trip distances (<=0, >70)')
# check the shape of the dataframe - how many rows were removed?
print(all_trips.shape)
print('Removed rows: ', current_length - all_trips.shape[0])
# in percent
print('Removed rows %: ', (current_length - all_trips.shape[0]) / current_length * 100, '%')

(39656098, 19)
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-01-01 00:35:40   2022-01-01 00:53:29              2.0   
1         1  2022-01-01 00:33:43   2022-01-01 00:42:07              1.0   
2         2  2022-01-01 00:53:21   2022-01-01 01:02:19              1.0   
3         2  2022-01-01 00:25:21   2022-01-01 00:35:23              1.0   
4         2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           3.80         1.0                  N           142           236   
1           2.10         1.0                  N           236            42   
2           0.97         1.0                  N           166           166   
3           1.09         1.0                  N           114            68   
4           4.30         1.0                  N            68           163   

   payment_type  fare_amount  extra  mta_tax  tip_amount  t

# Check location attributes - location ids (PULocationID, DOLocationID)
What location ids are used? Are there any missing ones?

In [3]:
# get a list of all used pickup and drop-off location ids
# use the unique() function
# sort the list
# convert the list to a dataframe

unique_pickup_location_ids = all_trips['PULocationID'].unique()
unique_pickup_location_ids.sort()
unique_pickup_location_ids = pd.DataFrame(unique_pickup_location_ids, columns=['PULocationID'])

unique_dropoff_location_ids = all_trips['DOLocationID'].unique()
unique_dropoff_location_ids.sort()
unique_dropoff_location_ids = pd.DataFrame(unique_dropoff_location_ids, columns=['DOLocationID'])

# which ones are missing? (between 1 and 263)
# use the set() function to get the difference between the two lists
# convert the result to a list
# sort the list
# convert the list to a dataframe
missing_pickup_location_ids = list(set(range(1, 264)) - set(unique_pickup_location_ids['PULocationID']))
missing_pickup_location_ids.sort()
missing_pickup_location_ids = pd.DataFrame(missing_pickup_location_ids, columns=['PULocationID'])

missing_dropoff_location_ids = list(set(range(1, 264)) - set(unique_dropoff_location_ids['DOLocationID']))
missing_dropoff_location_ids.sort()
missing_dropoff_location_ids = pd.DataFrame(missing_dropoff_location_ids, columns=['DOLocationID'])

# print the lists
print("##### missing_pickup_location_ids #####")
print(missing_pickup_location_ids)
print("##### missing_dropoff_location_ids #####")
print(missing_dropoff_location_ids)


##### unique_pickup_location_ids #####
     PULocationID
0               1
1               2
2               3
3               4
4               5
..            ...
257           261
258           262
259           263
260           264
261           265

[262 rows x 1 columns]
##### unique_dropoff_location_ids #####
     DOLocationID
0               1
1               2
2               3
3               4
4               5
..            ...
257           261
258           262
259           263
260           264
261           265

[262 rows x 1 columns]
##### missing_pickup_location_ids #####
   PULocationID
0           103
1           104
2           110
##### missing_dropoff_location_ids #####
   DOLocationID
0           103
1           104
2           199


# Clean location attributes (PULocationID, DOLocationID)

In [ ]:
# save current length of dataframe
current_length = all_trips.shape[0]

# filter trips with unknown location ids (pickup or drop-off)
# according to the data dictionary, the location ids are between 1 and 263, 264 is unknown and 265 is unknown

# filter trips with unknown pickup location id
all_trips = all_trips[all_trips['PULocationID'] < 264]
# filter trips with unknown drop-off location id
all_trips = all_trips[all_trips['DOLocationID'] < 264]

# check the shape of the dataframe - how many rows were removed?
print('after removing trips with unknown pickup or drop-off location id')
print(all_trips.shape)
print('Removed rows: ', current_length - all_trips.shape[0])
# in percent
print('Removed rows %: ', (current_length - all_trips.shape[0]) / current_length * 100, '%')

# save current length of dataframe
current_length = all_trips.shape[0]

# also filter trips with location id 0
all_trips = all_trips[all_trips['PULocationID'] > 0]
all_trips = all_trips[all_trips['DOLocationID'] > 0]

# check the shape of the dataframe - how many rows were removed?
print('after removing trips with location id 0')
print(all_trips.shape)
print('Removed rows: ', current_length - all_trips.shape[0])
# in percent
print('Removed rows %: ', (current_length - all_trips.shape[0]) / current_length * 100, '%')


# Quick overview of the data cleaning (trip_distance, PULocationID, DOLocationID)

In [ ]:
# total difference in rows removed
print('Total removed rows so far: ', original_data_length - all_trips.shape[0])
# in percent
print('Total removed rows %: ', (original_data_length - all_trips.shape[0]) / original_data_length * 100, '%')